**API WORLD BANK**

запросы и merge

In [14]:
!pip install pandas requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 3.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 3.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pandas]2m2/3 [pandas]

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip3 install --upgrade pip


In [15]:
import sys, os
sys.path.append(os.path.join(os.getcwd(), "log"))

from my_logger import setup_logging, get_logger
setup_logging()
log = get_logger("api_worldbank")
log.info("начало работы")

In [16]:
import pandas as pd
import requests

In [17]:
url = 'https://api.worldbank.org/v2/country?format=json'

Из API WORLDBANK берем следующие признаки:

In [18]:
gdp_ind = 'NY.GDP.MKTP.CD'
unemployment_ind = 'SL.UEM.TOTL.ZS'
tertiary_ed_attainment_ind = 'SE.TER.CUAT.BA.ZS'
enrollment_tert_ind = 'SE.TER.ENRR'
ad_literacy_rate_ind = 'SE.ADT.LITR.ZS'
ed_years_exp_ind = 'HD.HCI.EYRS'

**ДОБАВЛЕНИЕ ВВП**

In [26]:
url = f'https://api.worldbank.org/v2/country/all/indicator/{gdp_ind}?format=json&per_page=20000'
log.info(f'Данные ВВП по адресу: {url}')

#вводим учет повторов результата для одной страны, чтобы брать самые свежие значения показателя, отсекая другие
data = []
seen = set()

try:
    res = requests.get(url).json()[1]
except requests.exceptions.RequestException as e:
    log.error(f"Ошибка при запросе к API WorldBank: {e}")
    res = []

for i in res:
    val = i.get('value')
    iso3_code = i.get('countryiso3code')
    if val is None:
        continue
    if iso3_code in seen:
        continue

#добавляем столбцы с собранными данными
    data.append({
        'country_name': i['country']['value'],
        'iso2_code': i['country']['id'],
        'iso3_code': iso3_code,
        'gdp': val
    })
    seen.add(iso3_code)
    
log.info(f"Собрано уникальных стран с данными о ВВП: {len(data)}")

df_gdp = pd.DataFrame(data)

log.info(f"df_gdp создан: {df_gdp.shape[0]} строк, {df_gdp.shape[1]} столбцов")

df_gdp

,country_name,iso2_code,iso3_code,gdp
0,Africa Eastern and Southern,ZH,AFE,1.205974e+12
1,Africa Western and Central,ZI,AFW,6.700257e+11
2,Arab World,1A,ARB,3.704768e+12
3,Caribbean small states,S3,CSS,9.157080e+10
4,Central Europe and the Baltics,B8,CEB,2.455559e+12
...,...,...,...,...
254,Virgin Islands (U.S.),VI,VIR,4.672000e+09
255,West Bank and Gaza,PS,PSE,1.371110e+10
256,"Yemen, Rep.",YE,YEM,2.160616e+10
257,Zambia,ZM,ZMB,2.632578e+10


In [ ]:
df_gdp.to_csv('api_gdp_temp.csv', index=False)

**ДОБАВЛЕНИЕ БЕЗРАБОТИЦЫ**


In [27]:
url = f"https://api.worldbank.org/v2/country/all/indicator/{unemployment_ind}?format=json&per_page=20000"
log.info(f"Данные безработицы по адресу: {url}")

data = []
seen = set()

try:
    res = requests.get(url).json()[1]
except requests.exceptions.RequestException as e:
    log.error(f"Ошибка при запросе к API WorldBank: {e}")
    res = []

for i in res:
    val = i.get('value')
    iso3_code = i.get('countryiso3code')
    if val is None:
        continue
    if iso3_code in seen:
        continue

    data.append({
        'country_name': i['country']['value'],
        'iso2_code': i['country']['id'],
        'iso3_code': iso3_code,
        'unemployment_rate': val
    })
    seen.add(iso3_code)

log.info(f"Собрано уникальных стран с данными о безработице: {len(data)}")

df_unemployment_rate = pd.DataFrame(data)

log.info(f"df_unemployment_rate создан: {df_unemployment_rate.shape[0]} строк, {df_unemployment_rate.shape[1]} столбцов")

df_unemployment_rate

,country_name,iso2_code,iso3_code,unemployment_rate
0,Africa Eastern and Southern,ZH,AFE,7.772654
1,Africa Western and Central,ZI,AFW,3.218313
2,Arab World,1A,ARB,9.461238
3,Caribbean small states,S3,CSS,7.423939
4,Central Europe and the Baltics,B8,CEB,3.867636
...,...,...,...,...
227,Virgin Islands (U.S.),VI,VIR,12.014000
228,West Bank and Gaza,PS,PSE,24.420000
229,"Yemen, Rep.",YE,YEM,17.086000
230,Zambia,ZM,ZMB,5.961000


In [21]:
df_unemployent_rate.to_csv('api_unemployent_temp.csv', index=False)

**ДОБАВЛЯЕМ ОХВАТ ВЫСШИМ ОБРАЗОВАНИЕМ ОТ ВОЗРАСТНОЙ ГРУППЫ (School enrollment, tertiary (% gross))**

In [28]:
url = f'https://api.worldbank.org/v2/country/all/indicator/{enrollment_tert_ind}?format=json&per_page=20000'
log.info(f"Данные о зачислении в вузы (enrollment tertiary) по адресу: {url}")

data = []
seen = set()

try:
    res = requests.get(url).json()[1]
except requests.exceptions.RequestException as e:
    log.error(f"Ошибка при запросе к API WorldBank: {e}")
    res = []

for i in res:
    val = i.get('value')
    iso3_code = i.get('countryiso3code')
    if val is None:
        continue
    if iso3_code in seen:
        continue

    data.append({
        'country_name': i['country']['value'],
        'iso2_code': i['country']['id'],
        'iso3_code': iso3_code,
        'school_enrollment_tert': val
    })
    seen.add(iso3_code)

log.info(f"Собрано уникальных стран с данными о зачислении в вузы: {len(data)}")

df_enroll_tert = pd.DataFrame(data)

log.info(f"df_enroll_tert создан: {df_enroll_tert.shape[0]} строк, {df_enroll_tert.shape[1]} столбцов")

df_enroll_tert

,country_name,iso2_code,iso3_code,school_enrollment_tert
0,Africa Eastern and Southern,ZH,AFE,8.661860
1,Africa Western and Central,ZI,AFW,10.190260
2,Arab World,1A,ARB,35.080399
3,Caribbean small states,S3,CSS,22.445900
4,Central Europe and the Baltics,B8,CEB,68.538887
...,...,...,...,...
241,Virgin Islands (U.S.),VI,VIR,33.551350
242,West Bank and Gaza,PS,PSE,44.490462
243,"Yemen, Rep.",YE,YEM,8.806900
244,Zambia,ZM,ZMB,3.961597


In [ ]:
df_enroll_tert.to_csv('api_enrollment_tert_temp.csv', index=False)

**ДОБАВЛЯЕМ УРОВЕНЬ ГРАМОТНОСТИ ВЗРОСЛЫХ (Literacy rate, adult total (% of people ages 15 and above))**

In [29]:
url = f"https://api.worldbank.org/v2/country/all/indicator/{ad_literacy_rate_ind}?format=json&per_page=20000"
log.info(f"Данные по уровню грамотности взрослых по адресу: {url}")

data = []
seen = set()

try:
    res = requests.get(url).json()[1]
except requests.exceptions.RequestException as e:
    log.error(f"Ошибка при запросе к API WorldBank: {e}")
    res = []
    
for i in res:
    val = i.get('value')
    iso3_code = i.get('countryiso3code')
    if val is None:
        continue
    if iso3_code in seen:
        continue

    data.append({
        'country_name': i['country']['value'],
        'iso2_code': i['country']['id'],
        'iso3_code': iso3_code,
        'adult_literacy_rate': val
    })
    seen.add(iso3_code)
log.info(f"Собрано уникальных стран с данными о грамотности взрослых: {len(data)}")

df_literacy_rate = pd.DataFrame(data)

log.info(f"df_literacy_rate создан: {df_literacy_rate.shape[0]} строк, {df_literacy_rate.shape[1]} столбцов")

df_literacy_rate

,country_name,iso2_code,iso3_code,adult_literacy_rate
0,Africa Eastern and Southern,ZH,AFE,73.695129
1,Africa Western and Central,ZI,AFW,61.173969
2,Arab World,1A,ARB,78.986427
3,Caribbean small states,S3,CSS,91.018753
4,Central Europe and the Baltics,B8,CEB,99.500389
...,...,...,...,...
204,Viet Nam,VN,VNM,96.129997
205,West Bank and Gaza,PS,PSE,97.944980
206,"Yemen, Rep.",YE,YEM,37.090000
207,Zambia,ZM,ZMB,82.041806


In [ ]:
df_literacy_rate.to_csv('api_literacy_temp.csv', index=False)

**ОБЪЕДИНЕНИЕ:**

соединяем все показатели, но на данном этапе в датасете останутся не только страны, но и регионы

In [ ]:
df1 = df_gdp[['iso3_code', 'country_name', 'iso2_code', 'gdp']]
df2 = df_unemployent_rate[['iso3_code', 'unemployment_rate']]
df3 = df_enroll_tert[['iso3_code', 'school_enrollment_tert']]
df4 = df_literacy_rate[['iso3_code', 'adult_literacy_rate']]

api_df = df1.merge(df2, on='iso3_code', how='outer')
api_df = api_df.merge(df3, on='iso3_code', how='outer')
api_df = api_df.merge(df4, on='iso3_code', how='outer')

api_df

,iso3_code,country_name,iso2_code,gdp,unemployment_rate,school_enrollment_tert,adult_literacy_rate
0,,High income,XD,7.152300e+13,4.398870,82.035652,97.709610
1,ABW,Aruba,AW,3.648573e+09,NaN,14.065542,96.820000
2,AFE,Africa Eastern and Southern,ZH,1.205974e+12,7.772654,8.661860,73.695129
3,AFG,Afghanistan,AF,1.715223e+10,13.295000,10.854360,37.270000
4,AFW,Africa Western and Central,ZI,6.700257e+11,3.218313,10.190260,61.173969
...,...,...,...,...,...,...,...
256,XKX,Kosovo,XK,1.114860e+10,NaN,NaN,NaN
257,YEM,"Yemen, Rep.",YE,2.160616e+10,17.086000,8.806900,37.090000
258,ZAF,South Africa,ZA,4.002607e+11,33.168000,23.493769,91.150002
259,ZMB,Zambia,ZM,2.632578e+10,5.961000,3.961597,82.041806


Осуществим сбор всех стран, где изначально удалим различные регионы

Проверка на регион осуществляется:
1.   проверкой существует ли у страны столица (тк во многих регионах ее нет,

2.  проверкой имеет ли страна значение Aggregates, тк оно встерчается только у регионов.



In [32]:
url_countries = "https://api.worldbank.org/v2/country?format=json&per_page=600"
log.info(f"Загрузка списка стран по адресу: {url_countries}")

res = requests.get(url_countries).json()[1]
rows = []

for c in res:
    region_val = c.get('region', {}).get('value')
    capital_val = c.get('capitalCity')

    if region_val != "Aggregates" and capital_val:
        rows.append({
            'country_name': c['name'],
            'iso2': c['iso2Code'],
            'iso3': c['id']
        })

log.info(f"Собрано стран с валидными данными: {len(rows)}")

df_countries = pd.DataFrame(rows)

log.info(f"df_countries создан: {df_countries.shape[0]} строк, {df_countries.shape[1]} столбцов")

df_countries


,country_name,iso2,iso3
0,Aruba,AW,ABW
1,Afghanistan,AF,AFG
2,Angola,AO,AGO
3,Albania,AL,ALB
4,Andorra,AD,AND
...,...,...,...
206,Kosovo,XK,XKX
207,"Yemen, Rep.",YE,YEM
208,South Africa,ZA,ZAF
209,Zambia,ZM,ZMB


In [ ]:
df_countries.to_csv('api_counties_temp.csv', index=False)

Сопоставим получаившуюся информацию по настоящим странам с признаками, которые содержали и регионы, чтобы наконец от них избавиться

In [ ]:
base = df_countries.rename(columns={'iso3':'iso3_code', 'iso2':'iso2_code'})

api_df = base[['iso3_code','country_name','iso2_code']]
api_df = api_df.merge(df_gdp[['iso3_code','gdp']], on='iso3_code', how='left')
api_df = api_df.merge(df_unemployent_rate[['iso3_code','unemployment_rate']], on='iso3_code', how='left')
api_df = api_df.merge(df_enroll_tert[['iso3_code','school_enrollment_tert']], on='iso3_code', how='left')
api_df = api_df.merge(df_literacy_rate[['iso3_code','adult_literacy_rate']], on='iso3_code', how='left')
api_df

,iso3_code,country_name,iso2_code,gdp,unemployment_rate,school_enrollment_tert,adult_literacy_rate
0,ABW,Aruba,AW,3.648573e+09,NaN,14.065542,96.820000
1,AFG,Afghanistan,AF,1.715223e+10,13.295,10.854360,37.270000
2,AGO,Angola,AO,8.039694e+10,14.464,9.954907,66.239998
3,ALB,Albania,AL,2.717774e+10,10.250,81.998627,97.680000
4,AND,Andorra,AD,4.039844e+09,NaN,67.676768,NaN
...,...,...,...,...,...,...,...
206,XKX,Kosovo,XK,1.114860e+10,NaN,NaN,NaN
207,YEM,"Yemen, Rep.",YE,2.160616e+10,17.086,8.806900,37.090000
208,ZAF,South Africa,ZA,4.002607e+11,33.168,23.493769,91.150002
209,ZMB,Zambia,ZM,2.632578e+10,5.961,3.961597,82.041806


In [ ]:
api_df.to_csv('api_worldbank_dataset.csv', index=False)